ETABS DESIGN PIERS AS COLUMNS

Assumptions:
- ETABS model should be correct and run without errors
- User must determine and input ductility factor and number of levels in the plastic hinge zone
- Program will design the minimum bar size and/or spacing for the section to work for shear and flexure
- This may not necessarily comply with code minimum requirements. This is to be checked manually
- Tool is to design walls for ULS load combinations
- ETABS load combinations must be set up as follows:
- eq_env_1 -> Envelope of load combinations with unfactored earthquake load for flexural design
- eq_env_2 -> Envelope of load combinations with factored earthquake load for shear design
- wind_env -> Envelope of all ULS wind combinations

    

In [1]:
import sys
sys.path.append('C:\_Github\structural_engineering_toolbox')
from etabs_tools import etabs_api
from design_functions import pier_design_as_column, as3600_wall_design
import pandas as pd

<>:2: SyntaxWarning: invalid escape sequence '\_'
<>:2: SyntaxWarning: invalid escape sequence '\_'
C:\Users\zeyad.moustafa\AppData\Local\Temp\ipykernel_31336\3442906891.py:2: SyntaxWarning: invalid escape sequence '\_'
  sys.path.append('C:\_Github\structural_engineering_toolbox')


Define load combinations to be considered

In [2]:
# INPUT

# names of load cases

eq_env_1 = '(88) RS ULS ENV' # earthquake envelope unfactored EQ for moment design
eq_env_2 = '(88) RS ULS ENV SHEAR' # earthquake envelope factored EQ for amplified shear design
wind_env = '(88) WIND ULS ENV' # wind envelope
gravity_env = ' ' # gravity envelope

g = '(0) G'
q = '(0) Q'
rs = '(1) Response Spectrum mu=2'

load_cases = [eq_env_1, eq_env_2, wind_env]

# assumed building ductility factor
mu_sp = 2.6

# number of levels within plastic hinge zone
number_phz_levels = 3

# set min and max bar dia and spacing for wall reinforcement
v_bar_dia_min = 12
v_bar_dia_max = 28
v_bar_cts_min = 200
v_bar_cts_max = 200

h_bar_dia_min = 12
h_bar_dia_max = 20
h_bar_cts_min = 200
h_bar_cts_max = 200

Create instance of ETABS API and Sap Model object

In [3]:
etabs_api = etabs_api.etabs_api()
sap_model = etabs_api.sap_model

Get data from open ETABS model

In [4]:
story_data = etabs_api.get_story_data()
piers = etabs_api.get_piers(load_cases=load_cases)
pier_forces = etabs_api.get_pier_forces(load_cases=load_cases)

Print as Dataframe

In [5]:
piers_df = pd.DataFrame(piers)

Design one pier at a time

In [7]:
pier = piers[0]
pier['(88) RS ULS ENV Bottom Max']

{'Story Name': 'LEVEL 1',
 'Pier Name': 'P01',
 'Load Case': '(88) RS ULS ENV',
 'Step Type': 'Max',
 'Location': 'Bottom',
 'p': 371269.8801652028,
 'v2': 54685.367191293655,
 'v3': 6543.749544842902,
 't': 1434558.3552646325,
 'm2': 4336674.876742011,
 'm3': 122476646.57439846}

Get levels within plastic hinge zone

In [7]:
story_data = etabs_api.get_story_data()
story_names = etabs_api.get_story_names()

phz_levels = as3600_wall_design.get_phz_stories(story_names[0], number_phz_levels, story_names)
print(phz_levels)

['LEVEL 1', 'LEVEL 2', 'LEVEL 3', 'LEVEL 4']


Is pier within PHZ or not

In [8]:
is_within_phz = as3600_wall_design.is_in_phz(pier['Story Name'], phz_levels)
print(is_within_phz)

True


Test shear force

In [9]:
# shear forces top and bottom of section
eq_env_shear_top = round( pier[eq_env_2 + ' Top']['v2'] / 1000, 1 )
eq_env_shear_bot = round( pier[eq_env_2 + ' Bottom']['v2'] / 1000, 1 )

print(pier['Pier Name'])
print(eq_env_shear_top)
print(eq_env_shear_bot)

# if is_within_phz == True and mu_sp > 1.3:
#     v_design_top = g_v2_top + 

P01
-366.7
-337.3
